In [24]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
from mlxtend.preprocessing import TransactionEncoder

In [25]:
#### conectar_base_de_Datos

conn=sql.connect('data\\db_movies')
cur=conn.cursor()

## 2 Sistema de recomendación basado popularidad 


## 2.1 Sistema de recomendación basado en contenido KNN un solo producto visto 

In [26]:
df=pd.read_sql('select * from movie_final',conn)
df['movieId'] = df['movieId'].astype('object')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  134 non-null    object
 1   title    134 non-null    object
 2   genres   134 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,6,Heat (1995),Action|Crime|Thriller
3,10,GoldenEye (1995),Action|Adventure|Thriller
4,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [32]:
genres = df['genres'].str.split('|') # Divide la columna 'genres' en listas
te = TransactionEncoder() # Utiliza TransactionEncoder para convertirlo a variables dummy
genres_encoded = te.fit_transform(genres)
genres_df = pd.DataFrame(genres_encoded, columns=te.columns_) # Convierte el resultado en un dataframe con los nombres de las columnas correspondientes
df_dum = pd.concat([df, genres_df], axis=1) # Concatena el dataframe original con el dataframe de géneros codificados

# Elimina la columna original 'genres' y 'title'
df_dum.drop(columns=['genres'], inplace=True) 
df_dum.drop(columns=['title'], inplace=True)

# Muestra los primeros registros del dataframe resultante
df_dum.head()

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Drama,Fantasy,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,False,True,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False
1,2,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False
2,6,True,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False
3,10,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
4,32,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,False,False


In [40]:
model = neighbors.NearestNeighbors(n_neighbors=6, metric='cosine') 
model.fit(df_dum)
dist, idlist = model.kneighbors(df_dum)

In [41]:
distancias=pd.DataFrame(dist) ## devuelve un ranking de la distancias más cercanas para cada fila (pelicula)
id_list=pd.DataFrame(idlist) ## para saber esas distancias a que item corresponde

In [42]:
def MovieRecommender(movie_name = list(df['title'].value_counts().index)):
    movie_list_name = []
    movie_id = df[df['title'] == movie_name].index
    movie_id = movie_id[0]
    for newid in idlist[movie_id]:
        movie_list_name.append(df.loc[newid].title)
    return movie_list_name


print(interact(MovieRecommender))

interactive(children=(Dropdown(description='movie_name', options=('Toy Story (1995)', 'Truman Show, The (1998)…

<function MovieRecommender at 0x000001BCAE512340>
